In [1]:
import pandas as pd
# because the dataset is int tsv format we have to use delimeter.
df = pd.read_csv(r'C:\Users\ritth\code\Data\df_extracted.csv')

# creating a copy so we don't messed up our original dataset.
data=df.copy()
data.head(10)

,Unnamed: 0.1,Unnamed: 0,content,sponsored
0,1,1,It was a pleasure to welcome Jonnie Irwin and ...,0
1,2,2,"My staff seem to be on Facebook, Twitter and o...",0
2,4,4,,0
3,5,5,See our online help Get Support Here 2012 Al...,0
4,7,7,"obots"" content=""noindex, follow"" /> script typ...",0
5,8,8,"Nearly a year ago to the day,Brewmeister brewe...",0
6,9,9,Gadajace Glowy (Talking Heads) is a serious do...,0
7,10,10,A new study has come out that shows the Greek...,0
8,11,11,"BRASLIABrazils next finance minister, to be ap...",0
9,12,12,Follow and Subscribe,0


In [2]:
#data.drop(['Unnamed:0.1'],axis=1,inplace=True)
sentences=data.content.values
labels = data.sponsored.values
data.head()

,Unnamed: 0.1,Unnamed: 0,content,sponsored
0,1,1,It was a pleasure to welcome Jonnie Irwin and ...,0
1,2,2,"My staff seem to be on Facebook, Twitter and o...",0
2,4,4,,0
3,5,5,See our online help Get Support Here 2012 Al...,0
4,7,7,"obots"" content=""noindex, follow"" /> script typ...",0


In [3]:
from transformers import BertTokenizer
# using the low level BERT for our task.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Printing the original sentence.
print(' Original: ', sentences[0])

# Printing the tokenized sentence in form of list.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  It was a pleasure to welcome Jonnie Irwin and his crew down to the brewery this month to film a piece for BBC2 s Escape to the Country . In each episode presenter Jonnie helps a couple who are keen to relocate to somewhere more rural and lines up several possible houses to suit their... The building of the Otter Brewery was, and still is, a labour of love. We believe that s why our beer tastes so great and why more and more people are drinking Otter Beer. From old time drinkers and die-hard landlords to new age drinkers, we have tried to create a range of beers that provides something for everyone. We hope this website gives a small taste of what we are about. A pint of our beer would do just as well - if not better! We hope youenjoy both. 
Tokenized:  ['it', 'was', 'a', 'pleasure', 'to', 'welcome', 'jon', '##nie', 'irwin', 'and', 'his', 'crew', 'down', 'to', 'the', 'brewery', 'this', 'month', 'to', 'film', 'a', 'piece', 'for', 'bbc', '##2', 's', 'escape', 'to', 'the', 'cou

In [4]:
input_ids = []
for sent in sentences:
    # so basically encode tokenizing , mapping sentences to thier token ids after adding special tokens.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence which are encoding.
                        add_special_tokens = True, # Adding special tokens '[CLS]' and '[SEP]'

                         )
    
 
    input_ids.append(encoded_sent)

Token indices sequence length is longer than the specified maximum sequence length for this model (5871 > 512). Running this sequence through the model will result in indexing errors


In [5]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 128

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN , truncating="post", padding="post")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
attention_masks = []

for sent in input_ids:
    
    # Generating attention mask for sentences.
    #   - when there is 0 present as token id we are going to set mask as 0.
    #   - we are going to set mask 1 for all non-zero positive input id.
    att_mask = [int(token_id > 0) for token_id in sent]
    
   
    attention_masks.append(att_mask)

In [ ]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,test_size=0.1)

In [ ]:
import torch
#changing the numpy arrays into tensors for working on GPU. 
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Deciding the batch size for training.

batch_size = 32

#DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# DataLoader for our validation(test) set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig


model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,   
    output_attentions = False,
    output_hidden_states = False,
)


In [ ]:
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )
criterion = nn.BCELoss()

In [ ]:
import time
epochs = 1
#import format_time

In [ ]:
loss_values = []
test_losses = []
test_acc = []


for epoch_i in range(0, epochs):
    t0 = time.time()

    total_loss = 0
    running_test_loss = 0
    running_test_acc = 0
    # putting model in traing mode there are two model eval and train for model
    model.train()
    
    
    for step, batch in enumerate(train_dataloader):
        
        #checking time taken after every 50 steps.
        #if step % 50 == 0 and not step == 0:
            #elapsed = format_time(time.time() - t0)
        
        #getting ids,mask,labes for every batch
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_labels = batch[2]
        
        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

        loss = outputs[0]

        total_loss += loss.item()
        
        # doing back propagation
        loss.backward()

        optimizer.step()
        
        
    avg_train_loss = total_loss / len(train_dataloader) 
    loss_values.append(avg_train_loss)
    
    print(f'TrainLoss: {loss_values[-1] :.4f}')
    
   
 
    model.eval()
   
    with torch.no_grad():
        for batch_idx, data in enumerate(validation_dataloader):
            b_input_ids = batch[0]
            b_input_mask = batch[1]
            b_labels = batch[2]
            
            
            outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

            loss = outputs[0]

            running_test_loss += loss.item()

            # accuracy
            targets = targets.int().numpy()
            probs = torch.sigmoid(outputs).detach()
            preds = (probs >= 0.5).int().numpy()

            running_test_acc += accuracy_score(targets, preds)

        test_acc.append(running_test_acc / len(validation_dataloader))
        test_losses.append(running_test_loss / len(validation_dataloader))

        print(f'TestLoss: {test_losses[-1] :.4f}')
        print(f'TestAccu: {test_acc[-1] *100 :.2f}%')
        print(f'Time: {time.time() - t0 :.4f}')


        # save best model
        if test_acc[-1] > benchmark_acc:
            # save model to cpu
            torch.save(model.state_dict(), './model.pth')
            

            # update benckmark
            benchmark_acc = test_acc[-1]



plt.plot(loss_values, label= "Train Loss")
plt.plot(test_losses, label= "Test Loss")
plt.xlabel(" Iteration ")
plt.ylabel("Loss value")
plt.legend(loc="upper left")
plt.show()
